In [1]:
# importing the necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
from datetime import datetime as dt

In [2]:
# loading the data frame and converting attiributes to object type or datetime.
listings = pd.read_csv('../data/listings_clean.csv')
# converting data type to datetime.
listings[['host_since', 'calendar_last_scraped','first_review','last_review']] = listings[['host_since', 'calendar_last_scraped','first_review','last_review']].apply(pd.to_datetime)
# changing data type of is,host_id columns to object
listings[['id','host_id']] = listings[['id','host_id']].astype('object')
listings.select_dtypes('object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6883 entries, 0 to 6882
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      6883 non-null   object
 1   host_id                 6883 non-null   object
 2   host_response_time      5021 non-null   object
 3   host_verifications      6883 non-null   object
 4   neighbourhood           4938 non-null   object
 5   neighbourhood_cleansed  6883 non-null   object
 6   property_type           6883 non-null   object
 7   room_type               6883 non-null   object
 8   amenities               6883 non-null   object
 9   license                 3878 non-null   object
dtypes: object(10)
memory usage: 537.9+ KB


There are 10 columns that are in object type. I will keep id and host_id columns as object type. I looked into the remaining columns and here what I will do for them:

1. host_response_time will be encoded using ordinal encoder. 
2. host_verifications will be encoded using get_dummies.
3. neighbourhood column will be dropped since it contains cities and I don't think we can get useful information from that.
4. neighbourhood_cleansed column will be renamed as neighbourhood and will be encoded using get_dummies.
5. property_type column will be dropped since it contains subgroups of room_type such as entire_home_bungalow etc. 
6. room_type will be encoded using get_dummies.
7. amenities will be encoded using MultiLabelBinarizer. I need to further curate the amenities to reduce the number of labels.
8. A new feature will be created from host_since column as years_as_host.

In [3]:
# checking the unique values in the host_response_time.
listings.host_response_time.unique()

array(['within an hour', 'within a few hours', nan, 'within a day',
       'a few days or more'], dtype=object)

* I can order these entries as:  
within an hour < within a few hours < within a day < a few days or more

So, I can use ordinal encoder for this variable.

QUESTION: Should I encode it other way around since it would be better to get response quickly?

In [4]:
response_time_dict = {'within an hour':1, 'within a few hours':2, 'within a day':3, 'a few days or more':4 }
listings['host_response_time_ordinal'] = listings.host_response_time.map(response_time_dict)
listings = listings.drop('host_response_time',axis=1)

In [5]:
# let's check the host_verifications column.
listings[['host_verifications']].head()

,host_verifications
0,"['email', 'phone', 'facebook', 'reviews', 'kba']"
1,"['email', 'phone', 'reviews', 'kba', 'work_ema..."
2,"['email', 'phone', 'reviews', 'jumio', 'govern..."
3,"['email', 'phone', 'reviews', 'jumio', 'govern..."
4,"['email', 'phone', 'reviews', 'kba']"


Host verifications column contains a list of verification methods. First of all I need to check if each row contains a list or is it string. 

In [6]:
print(type(listings['host_verifications'][0]))
print(listings['host_verifications'][0])

<class 'str'>
['email', 'phone', 'facebook', 'reviews', 'kba']


I need to split each of these verification methods and create a list out of them. I will use comma as splitting criteria, but before that I need to get rid of all non-letter characters except comma.

In [7]:
for i in range(listings .shape[0]):
    listings .host_verifications[i] = listings.host_verifications[i].replace('[','').replace("'",'').replace(']','').replace(' ','').split(',')

print(listings['host_verifications'][0])
print(type(listings['host_verifications'][0]))

/Users/dagdasy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['email', 'phone', 'facebook', 'reviews', 'kba']
<class 'list'>


I will convert these into labels using MultiLabelBinarizer.

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

listings = listings.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(listings.pop('host_verifications')),
                index=listings.index,
                columns='host_ver_' + mlb.classes_))

In [9]:
# lets check the neighbourhood column
listings.neighbourhood.unique()

array(['San Francisco, California, United States', nan,
       'San Francisco, Hayes Valley, California, United States',
       'Noe Valley - San Francisco, California, United States',
       'Daly City, California, United States',
       'San Franscisco, California, United States',
       'San Jose, California, United States',
       'San Francisco , Ca, United States'], dtype=object)

neighbourhood column is too broad. I will check the neighbourhood_cleansed column. If there is detailed information in it then I will drop the first and use the latter.

In [10]:
# lets check the neighbourhood_cleansed column
listings.neighbourhood_cleansed.unique()

array(['Western Addition', 'Bernal Heights', 'Haight Ashbury', 'Mission',
       'Nob Hill', 'Downtown/Civic Center', 'Castro/Upper Market',
       'Noe Valley', 'Twin Peaks', 'Potrero Hill', 'Outer Richmond',
       'Glen Park', 'Ocean View', 'Pacific Heights', 'Financial District',
       'Inner Sunset', 'Russian Hill', 'Outer Sunset', 'Marina',
       'Inner Richmond', 'Excelsior', 'Seacliff', 'West of Twin Peaks',
       'Bayview', 'North Beach', 'Presidio', 'Presidio Heights',
       'Diamond Heights', 'Chinatown', 'Parkside', 'South of Market',
       'Outer Mission', 'Lakeshore', 'Crocker Amazon', 'Golden Gate Park',
       'Visitacion Valley'], dtype=object)

In [11]:
listings = listings.drop('neighbourhood',axis=1)

neighbourhood_cleansed column has detailed information that could separate "good" neighbourhoods from the "bad" ones which could impact the pricing of the listings.
I will use get_dummies to create the labels.

In [12]:
listings = pd.get_dummies(listings, prefix='neighbourhood', prefix_sep='_', dummy_na=False, columns=['neighbourhood_cleansed'], sparse=False, drop_first=False, dtype=None)

In [13]:
# let's check the property_type column.

listings.property_type.unique()

array(['Entire apartment', 'Private room in apartment',
       'Entire condominium', 'Private room in condominium',
       'Entire house', 'Private room in townhouse', 'Entire loft',
       'Private room in house', 'Entire guest suite', 'Entire cottage',
       'Private room in cottage', 'Shared room in hostel',
       'Private room in hostel', 'Private room in guest suite',
       'Entire guesthouse', 'Shared room in house', 'Entire bungalow',
       'Entire townhouse', 'Shared room in apartment',
       'Shared room in loft', 'Private room in bed and breakfast',
       'Entire serviced apartment', 'Room in hotel',
       'Room in boutique hotel', 'Shared room in bed and breakfast',
       'Private room in loft', 'Room in serviced apartment',
       'Private room in villa', 'Barn', 'Room in aparthotel',
       'Entire floor', 'Private room in resort',
       'Private room in serviced apartment', 'Private room in bungalow',
       'Shared room in townhouse', 'Private room', 'Entire pla

This looks like a subset of room_type column. I will drop this column completely and use room_type information for the model.


In [14]:
# let's check the room_type column.

listings.room_type.unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [15]:
listings = pd.get_dummies(listings, prefix='room_type', prefix_sep='_', dummy_na=False, columns=['room_type'], sparse=False, drop_first=False, dtype=None)

In [16]:
listings = listings.drop('property_type', axis=1)

In [17]:
# let's check the amenities column.
listings[['amenities']].head()

,amenities
0,"[""Hangers"", ""Essentials"", ""Oven"", ""Free street..."
1,"[""Essentials"", ""Wifi"", ""Private entrance"", ""De..."
2,"[""Lock on bedroom door"", ""Hot water"", ""Private..."
3,"[""Lock on bedroom door"", ""Hot water"", ""Private..."
4,"[""Free street parking"", ""Hair dryer"", ""Heating..."


* I will repeat same procedure I used for host_verifications for the amenities column.

In [18]:
df_amenities = listings[['amenities']]
for i in range(df_amenities .shape[0]):
    df_amenities .amenities[i] = df_amenities .amenities[i].replace('[','').replace('"','').replace(']','').split(',')

print(df_amenities['amenities'][0])
print(type(df_amenities['amenities'][0]))

/Users/dagdasy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2878: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


['Hangers', ' Essentials', ' Oven', ' Free street parking', ' Coffee maker', ' Cable TV', ' Pack \\u2019n Play/travel crib', ' TV', ' Refrigerator', ' Dedicated workspace', ' Room-darkening shades', ' Kitchen', ' Washer', ' Long term stays allowed', ' Heating', ' Microwave', ' Dryer', ' Keypad', ' Shampoo', ' Iron', ' Garden or backyard', ' Hot water', ' Stove', ' Wifi', ' First aid kit', ' Carbon monoxide alarm', ' Hair dryer', ' Dishes and silverware', ' Smoke alarm', ' Cooking basics', ' Private entrance']
<class 'list'>


In [19]:
mlb = MultiLabelBinarizer(sparse_output=True)

df_amenities = df_amenities.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df_amenities.pop('amenities')),
                index=df_amenities.index,
                columns=mlb.classes_))

In [20]:
df_amenities.head()

,,1 space,120\ TV with Amazon Prime Video,28\ HDTV with Amazon Prime Video,3 Minute Miracle Moist conditioner,32\ HDTV with Amazon Prime Video,32\ TV,37\ HDTV with Netflix,4 burners and double oven electronic control stainless steel gas stove,40\ HDTV with Roku,...,Sound system with aux,Stove,TV,TV with Chromecast,TV with Netflix,Toaster,Washer,Washer \u2013\u00a0In building,Wifi,Wifi \u2013 1000 Mbps
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* There are 600 amenities. Are they really different or are they very same amenities but are being recognized as different due to slight changes in string

In [21]:
set(df_amenities.columns)

{'',
 ' 1 space',
 ' 120\\ TV with Amazon Prime Video',
 ' 28\\ HDTV with Amazon Prime Video',
 ' 3 Minute Miracle Moist conditioner',
 ' 32\\ HDTV with Amazon Prime Video',
 ' 32\\ TV',
 ' 37\\ HDTV with Netflix',
 ' 4 burners and double oven electronic control stainless steel gas stove',
 ' 40\\ HDTV with Roku',
 ' 40\\ TV',
 ' 42\\ HDTV with Apple TV',
 ' 42\\ HDTV with Chromecast',
 ' 42\\ HDTV with HBO Max',
 ' 42\\ HDTV with Netflix',
 ' 42\\ HDTV with premium cable',
 ' 43\\ HDTV',
 ' 44\\ HDTV with Amazon Prime Video',
 ' 46\\ HDTV',
 ' 46\\ HDTV with Netflix',
 ' 47\\ HDTV with Netflix',
 ' 49\\ HDTV with Chromecast',
 ' 5-10 years old',
 ' 50\\ HDTV',
 ' 50\\ HDTV with Netflix',
 ' 50\\ HDTV with Roku',
 ' 52\\ HDTV with standard cable',
 ' 55\\ HDTV',
 ' 55\\ HDTV with Amazon Prime Video',
 ' 55\\ HDTV with Chromecast',
 ' 55\\ HDTV with Netflix',
 ' 55\\ HDTV with Roku',
 ' 65\\ HDTV',
 ' 65\\ HDTV with Amazon Prime Video',
 ' 65\\ HDTV with Chromecast',
 ' 65\\ HDTV with N

* QUESTION: There are many amenities that could be aggregated into one. How should I do?

In [22]:
# Calculating how many years have passed as a host.
listings['years_as_host'] = round((pd.Timestamp.now().normalize() - listings['host_since'])/np.timedelta64(1,'Y'),1)

In [25]:
set(listings.columns)

{'accommodates',
 'amenities',
 'availability_30',
 'availability_365',
 'availability_60',
 'availability_90',
 'bathrooms',
 'bedrooms',
 'beds',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'calendar_last_scraped',
 'first_review',
 'has_availability',
 'host_acceptance_rate',
 'host_has_profile_pic',
 'host_id',
 'host_identity_verified',
 'host_is_superhost',
 'host_listings_count',
 'host_response_rate',
 'host_response_time_ordinal',
 'host_since',
 'host_total_listings_count',
 'host_ver_None',
 'host_ver_email',
 'host_ver_facebook',
 'host_ver_google',
 'host_ver_government_id',
 'host_ver_identity_manual',
 'host_ver_jumio',
 'host_ver_kba',
 'host_ver_manual_offline',
 'host_ver_manual_online',
 'host_ver_offline_government_id',
 'host_ver_phone',
 'host_ver_reviews',
 'host_ver_selfie',
 'host_ver_sent_id',
 'host_ver_work_email',
 'host_v